# Olo AISQL Functions Demo
## Unlocking AI-Powered Insights in the Hospitality Sector

This notebook demonstrates the power of Snowflake's AISQL functions using Olo's restaurant and delivery platform data. We'll explore how AI can transform raw operational data into actionable business insights for the hospitality industry.

### AISQL Functions Covered:
- **AI_SENTIMENT**: Analyze customer satisfaction and feedback sentiment
- **AI_CLASSIFY**: Categorize menu items, customer preferences, and operational data
- **AI_EXTRACT**: Extract specific insights from restaurant descriptions and reviews
- **AI_AGG**: Aggregate insights across multiple data points
- **AI_SUMMARIZE_AGG**: Generate executive summaries of business performance
- **AI_EMBED & AI_SIMILARITY**: Power recommendation engines for menu items
- **AI_COMPLETE**: Generate marketing content and descriptions

### Business Context:
Olo operates in the competitive hospitality sector, connecting restaurants with customers through digital ordering and delivery platforms. Understanding customer preferences, optimizing menu offerings, and improving operational efficiency are critical for success.


## Environment Setup

**Prerequisites:** 
1. Run `01_Olo_Setup_DataLoad.ipynb` first to set up the database and load all data
2. Upload `customer_reviews.csv` to your `@olo_data_stage` (this file contains realistic customer reviews for AI analysis)

Let's connect to the existing Olo demo environment.


In [ ]:
-- Connect to the existing Olo demo environment
USE DATABASE OLO_AISQL_DEMO;
USE SCHEMA RESTAURANT_DATA;
USE WAREHOUSE COMPUTE_WH; -- Adjust to your warehouse

-- Verify our environment and data
SELECT CURRENT_DATABASE() as database_name, CURRENT_SCHEMA() as schema_name;


In [ ]:
-- Verify data is loaded and ready for AISQL demonstrations
SELECT 'Restaurants' as table_name, COUNT(*) as records FROM restaurants
UNION ALL
SELECT 'Customers', COUNT(*) FROM customers
UNION ALL
SELECT 'Menu Items', COUNT(*) FROM menu_items
UNION ALL
SELECT 'Orders', COUNT(*) FROM orders
ORDER BY table_name;


## 1. AI_SENTIMENT: Customer Satisfaction Analysis

Let's start by analyzing customer sentiment. We'll create synthetic customer reviews and use AI_SENTIMENT to understand satisfaction levels across different restaurants and cuisine types.


In [ ]:
-- Create customer reviews table and load from CSV
CREATE OR REPLACE TABLE customer_reviews (
    review_id INT PRIMARY KEY,
    order_id INT,
    customer_id INT,
    restaurant_id INT,
    restaurant_name VARCHAR(100),
    cuisine_type VARCHAR(50),
    review_text TEXT,
    rating INT,
    review_date DATE
);

-- Load customer reviews data from CSV
COPY INTO customer_reviews FROM @olo_data_stage/customer_reviews.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);

-- Verify the reviews loaded correctly
SELECT COUNT(*) as total_reviews FROM customer_reviews;

-- Show sample data to confirm it loaded
SELECT restaurant_name, cuisine_type, rating, review_text 
FROM customer_reviews 
LIMIT 5;

-- Test AI_SENTIMENT on one review to verify it works
SELECT 
    restaurant_name,
    LEFT(review_text, 50) || '...' as review_preview,
    AI_SENTIMENT(review_text) as full_sentiment_result,
    AI_SENTIMENT(review_text):categories[0]:sentiment as sentiment_label
FROM customer_reviews 
LIMIT 3;


In [ ]:
-- Comprehensive sentiment analysis: Individual reviews + Restaurant summary
WITH individual_sentiment AS (
    SELECT 
        restaurant_id,
        restaurant_name,
        cuisine_type,
        review_text,
        rating,
        review_date,
        AI_SENTIMENT(review_text):categories[0]:sentiment as sentiment_label,
        CASE 
            WHEN AI_SENTIMENT(review_text):categories[0]:sentiment = 'positive' THEN 1
            WHEN AI_SENTIMENT(review_text):categories[0]:sentiment = 'neutral' THEN 0
            WHEN AI_SENTIMENT(review_text):categories[0]:sentiment = 'negative' THEN -1
            ELSE 0
        END as sentiment_score
    FROM customer_reviews
)
-- Show individual reviews with sentiment
SELECT 
    restaurant_name,
    cuisine_type,
    LEFT(review_text, 80) || '...' as review_preview,
    sentiment_label,
    sentiment_score,  -- This shows -1, 0, or 1
    rating as original_star_rating,  -- Original 1-5 star rating for comparison
    review_date
FROM individual_sentiment
WHERE review_date >= '2024-01-01'
ORDER BY review_date DESC
LIMIT 12;


In [ ]:
-- Restaurant sentiment summary + Geographic analysis
WITH sentiment_base AS (
    SELECT 
        cr.restaurant_id,
        cr.restaurant_name,
        cr.cuisine_type,
        r.city,
        r.state,
        AI_SENTIMENT(cr.review_text):categories[0]:sentiment as sentiment_label,
        CASE 
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'positive' THEN 1
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'neutral' THEN 0
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'negative' THEN -1
            ELSE 0
        END as sentiment_score
    FROM customer_reviews cr
    JOIN restaurants r ON cr.restaurant_id = r.restaurant_id
)
-- Restaurant-level sentiment analysis
SELECT
    restaurant_name,
    cuisine_type,
    city || ', ' || state as location,
    COUNT(*) as total_reviews,
    COUNT(CASE WHEN sentiment_label = 'positive' THEN 1 END) as positive_reviews,
    COUNT(CASE WHEN sentiment_label = 'negative' THEN 1 END) as negative_reviews,
    ROUND((COUNT(CASE WHEN sentiment_label = 'positive' THEN 1 END) * 100.0 / COUNT(*)), 1) as positive_percentage,
    ROUND(AVG(sentiment_score), 3) as avg_sentiment_score
FROM sentiment_base
GROUP BY restaurant_name, cuisine_type, city, state
ORDER BY avg_sentiment_score DESC, positive_percentage DESC;

-- Business Insight: This comprehensive analysis shows restaurant performance with location context,
-- enabling both operational improvements and geographic expansion decisions.


## 2. AI_CLASSIFY: Menu Item and Customer Preference Classification

Next, we'll use AI_CLASSIFY to categorize menu items by dietary preferences and classify customer ordering patterns.


In [ ]:
-- IMPROVED: Classify menu items using item name + description for better accuracy
SELECT 
    m.item_name,
    m.description,
    r.cuisine_type,
    m.price,
    -- Use CONCAT to give AI_CLASSIFY full context (item name + description)
    AI_CLASSIFY(
        CONCAT(m.item_name, ': ', m.description),
        ['Meat-Based', 'Vegetarian', 'Vegan', 'Seafood', 'Mixed']
    ) as dietary_classification,
    AI_CLASSIFY(
        CONCAT(m.item_name, ': ', m.description),
        ['Spicy', 'Mild', 'Creamy', 'Grilled', 'Fried']
    ) as preparation_style
FROM menu_items m
JOIN restaurants r ON m.restaurant_id = r.restaurant_id
ORDER BY m.popularity_score DESC
LIMIT 20;

-- Business Insight: Using item name + description gives AI_CLASSIFY much better context.
-- Now "Chicken Tikka Masala: Creamy tomato curry..." will correctly classify as "Meat-Based".


In [ ]:
-- Classify customer ordering behavior patterns
WITH customer_behavior AS (
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name as customer_name,
        c.loyalty_tier,
        c.preferred_cuisine,
        c.total_orders,
        c.total_spent,
        ROUND(c.total_spent / c.total_orders, 2) as avg_order_value,
        CASE 
            WHEN c.total_orders >= 50 THEN 'Frequent orderer with high engagement and loyalty'
            WHEN c.total_orders >= 20 THEN 'Regular customer with consistent ordering patterns'
            WHEN c.total_orders >= 10 THEN 'Occasional customer with moderate engagement'
            ELSE 'New or infrequent customer with limited order history'
        END as customer_description
    FROM customers c
)
SELECT 
    customer_name,
    loyalty_tier,
    preferred_cuisine,
    total_orders,
    total_spent,
    avg_order_value,
    AI_CLASSIFY(
        customer_description,
        ['VIP Customer', 'Loyal Regular', 'Growing Customer', 'At-Risk Customer', 'New Customer']
    ) as customer_segment
FROM customer_behavior
ORDER BY total_spent DESC
LIMIT 15;

-- Business Insight: Customer segmentation enables targeted marketing campaigns 
-- and personalized experiences to increase retention and lifetime value.


## 3. AI_EXTRACT: Extracting Operational Insights

We'll use AI_EXTRACT to pull specific insights from restaurant descriptions and operational data.


In [ ]:
-- Create enhanced restaurant profiles with detailed descriptions
CREATE OR REPLACE TABLE restaurant_profiles AS
SELECT 
    r.*,
    CASE r.cuisine_type
        WHEN 'Italian' THEN 'Authentic Italian cuisine featuring handmade pasta, wood-fired pizzas, and traditional recipes passed down through generations. Known for using imported ingredients and creating a warm, family-friendly atmosphere.'
        WHEN 'Chinese' THEN 'Traditional Chinese restaurant offering a wide variety of regional dishes including Szechuan spicy specialties, Cantonese dim sum, and fresh wok-fried entrees. Emphasis on fresh vegetables and authentic cooking techniques.'
        WHEN 'American' THEN 'Classic American comfort food featuring gourmet burgers, craft beer selection, and locally-sourced ingredients. Casual dining atmosphere with focus on quality and generous portions.'
        WHEN 'Mexican' THEN 'Vibrant Mexican cuisine with fresh salsas, house-made tortillas, and authentic spices. Features both traditional favorites and modern fusion dishes with emphasis on fresh, local ingredients.'
        WHEN 'Japanese' THEN 'Premium Japanese dining experience specializing in fresh sushi, sashimi, and traditional cooked dishes. Commitment to authentic preparation methods and highest quality fish and ingredients.'
        WHEN 'BBQ' THEN 'Authentic barbecue smokehouse featuring slow-smoked meats, house-made sauces, and traditional Southern sides. Known for tender, flavorful meats and authentic pit-smoking techniques.'
        WHEN 'Vegetarian' THEN 'Plant-based cuisine focusing on organic, locally-sourced ingredients. Creative vegetarian and vegan options that are both nutritious and flavorful, appealing to health-conscious diners.'
        WHEN 'Vietnamese' THEN 'Traditional Vietnamese cuisine featuring fresh herbs, aromatic broths, and authentic pho recipes. Emphasis on healthy, light dishes with complex flavors and fresh ingredients.'
        ELSE 'Diverse menu offering quality dishes with focus on fresh ingredients and customer satisfaction.'
    END as restaurant_description
FROM restaurants r;


In [ ]:
-- Extract key selling points and operational strengths from restaurant descriptions
-- Using AI_EXTRACT with proper responseFormat syntax
SELECT 
    restaurant_name,
    cuisine_type,
    LEFT(restaurant_description, 100) || '...' as description_preview,
    AI_EXTRACT(
        text => restaurant_description,
        responseFormat => {'advantages': 'What are the main competitive advantages mentioned?', 'quality': 'What quality claims are made?', 'atmosphere': 'What type of dining atmosphere is described?'}
    ) as extracted_insights
FROM restaurant_profiles
LIMIT 5;

-- Business Insight: This analysis helps identify what restaurants emphasize 
-- in their positioning and can inform marketing strategies and competitive analysis.


In [ ]:
-- User-friendly view of extracted restaurant insights (optimized to call AI_EXTRACT once)
WITH extracted_insights AS (
    SELECT 
        restaurant_name,
        cuisine_type,
        AI_EXTRACT(
            text => restaurant_description,
            responseFormat => {'advantages': 'What are the main competitive advantages mentioned?', 'quality': 'What quality claims are made?', 'atmosphere': 'What type of dining atmosphere is described?'}
        ) as insights
    FROM restaurant_profiles
    LIMIT 8
)
SELECT 
    restaurant_name,
    cuisine_type,
    -- Parse the JSON response into readable columns
    insights:response:advantages as "🏆 Competitive Advantages",
    insights:response:quality as "✨ Quality Claims", 
    insights:response:atmosphere as "🍽️ Dining Atmosphere"
FROM extracted_insights
ORDER BY restaurant_name;

-- This provides a clean, structured view of the key insights extracted from each restaurant's description
-- Each column shows a specific aspect that can inform marketing and competitive positioning

## 4. AI_AGG & AI_SUMMARIZE_AGG: Aggregated Business Intelligence

Let's use AI_AGG to generate insights across multiple data points and AI_SUMMARIZE_AGG for executive summaries.


In [ ]:
-- Use AI_AGG to analyze customer feedback patterns across cuisine types
SELECT 
    cuisine_type,
    COUNT(*) as review_count,
    AI_AGG(
        review_text,
        'What are the most common positive aspects mentioned in these restaurant reviews?'
    ) as common_positives,
    AI_AGG(
        review_text,
        'What are the main areas for improvement mentioned in these reviews?'
    ) as improvement_areas
FROM customer_reviews
GROUP BY cuisine_type
ORDER BY review_count DESC;

-- Business Insight: This aggregated analysis reveals cuisine-specific 
-- strengths and weaknesses, enabling targeted operational improvements.


In [ ]:
-- Generate executive summary of customer satisfaction by cuisine type using AI_SUMMARIZE_AGG
SELECT 
    cuisine_type,
    COUNT(*) as total_reviews,
    ROUND(AVG(CASE 
        WHEN AI_SENTIMENT(review_text):categories[0]:sentiment = 'positive' THEN 1
        WHEN AI_SENTIMENT(review_text):categories[0]:sentiment = 'neutral' THEN 0
        WHEN AI_SENTIMENT(review_text):categories[0]:sentiment = 'negative' THEN -1
        ELSE 0
    END), 3) as avg_sentiment,
    AI_SUMMARIZE_AGG(review_text) as executive_summary
FROM customer_reviews
GROUP BY cuisine_type
ORDER BY avg_sentiment DESC;

-- Business Insight: These summaries provide executives with quick insights 
-- into customer perception and satisfaction across different cuisine segments.


In [ ]:
-- Analyze menu item descriptions to identify trending ingredients and preparation methods
SELECT 
    category,
    COUNT(*) as item_count,
    ROUND(AVG(price), 2) as avg_price,
    AI_AGG(
        description,
        'What cooking methods and preparation techniques are most commonly mentioned?'
    ) as common_cooking_methods,
    AI_AGG(
        description,
        'What ingredients or flavor profiles appear most frequently?'
    ) as popular_ingredients
FROM menu_items
GROUP BY category
HAVING COUNT(*) >= 3
ORDER BY avg_price DESC;

-- Business Insight: Understanding trending ingredients and cooking methods 
-- helps restaurants stay competitive and develop new menu items.


## 5. AI_COMPLETE: Marketing Content Generation

Use AI_COMPLETE to generate marketing content, promotional descriptions, and customer communications at scale.


In [ ]:
-- Generate promotional content for top-performing restaurants
WITH top_restaurants AS (
    SELECT 
        r.restaurant_name,
        r.cuisine_type,
        COUNT(o.order_id) as order_count,
        ROUND(AVG(o.total_amount), 2) as avg_order_value
    FROM restaurants r
    JOIN orders o ON r.restaurant_id = o.restaurant_id
    WHERE o.status = 'delivered'
    GROUP BY r.restaurant_name, r.cuisine_type
    ORDER BY order_count DESC
    LIMIT 5
)
SELECT 
    restaurant_name,
    cuisine_type,
    order_count,
    avg_order_value,
    AI_COMPLETE(
        'llama3-8b',
        'Write a compelling promotional email subject line for ' || restaurant_name || 
        ', a popular ' || cuisine_type || ' restaurant with an average order value of $' || avg_order_value ||
        '. Make it engaging and highlight their popularity.'
    ) as email_subject_line,
    AI_COMPLETE(
        'llama3-8b',
        'Create a 50-word promotional description for ' || restaurant_name || 
        ', highlighting their ' || cuisine_type || ' cuisine and popularity among customers. ' ||
        'Mention they have served ' || order_count || ' orders. Make it appetizing and compelling.'
    ) as promotional_description
FROM top_restaurants;

-- Business Insight: AI-generated marketing content scales promotional efforts 
-- and ensures consistent, engaging messaging across restaurant partners.


In [ ]:
-- Generate personalized customer retention messages
WITH customer_insights AS (
    SELECT 
        c.customer_id,
        c.first_name,
        c.loyalty_tier,
        c.preferred_cuisine,
        c.total_orders,
        c.total_spent,
        DATEDIFF(days, c.last_order_date, CURRENT_DATE()) as days_since_last_order
    FROM customers c
    WHERE days_since_last_order > 30  -- Customers who haven't ordered recently
    ORDER BY total_spent DESC
    LIMIT 5
)
SELECT 
    customer_id,
    first_name,
    loyalty_tier,
    preferred_cuisine,
    total_spent,
    days_since_last_order,
    AI_COMPLETE(
        'llama3-8b',
        'Write a personalized retention email for ' || first_name || 
        ', a ' || loyalty_tier || ' customer who prefers ' || preferred_cuisine || 
        ' cuisine and has spent $' || total_spent || ' but hasn''t ordered in ' || 
        days_since_last_order || ' days. Make it warm and include a special offer.'
    ) as retention_message
FROM customer_insights;

-- Business Insight: Personalized retention campaigns powered by AI_COMPLETE 
-- can significantly improve customer lifetime value and reduce churn.


In [ ]:
-- Create personalized recommendations based on customer preferences
WITH customer_preference_embedding AS (
    SELECT 
        customer_id,
        first_name || ' ' || last_name as customer_name,
        preferred_cuisine,
        loyalty_tier,
        AI_EMBED('snowflake-arctic-embed-l-v2.0',
            'Customer prefers ' || preferred_cuisine || ' cuisine, ' ||
            loyalty_tier || ' tier member with ' || total_orders || ' orders'
        ) as preference_embedding
    FROM customers
    WHERE customer_id = 2001  -- Example customer
)
SELECT 
    c.customer_name,
    c.preferred_cuisine,
    c.loyalty_tier,
    m.item_name,
    m.restaurant_name,
    m.cuisine_type,
    m.price,
    ROUND(AI_SIMILARITY(m.description_embedding, c.preference_embedding), 4) as recommendation_score
FROM menu_embeddings m
CROSS JOIN customer_preference_embedding c
ORDER BY recommendation_score DESC
LIMIT 15;

-- Business Insight: Personalized recommendations increase order value 
-- and customer satisfaction by suggesting items aligned with individual preferences.


## Fix for AI_SENTIMENT AVG Error

The error occurs because `AI_SENTIMENT` returns a JSON object, not a numeric value. Here's the corrected query:

```sql
-- CORRECTED: Comprehensive restaurant performance analysis
WITH restaurant_analysis AS (
    SELECT 
        r.restaurant_name,
        r.cuisine_type,
        COUNT(DISTINCT o.order_id) as total_orders,
        ROUND(AVG(o.total_amount), 2) as avg_order_value,
        ROUND(SUM(o.total_amount), 2) as total_revenue,
        COUNT(DISTINCT o.customer_id) as unique_customers,
        -- FIXED: Extract sentiment score from JSON object
        ROUND(AVG(CASE 
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'positive' THEN 1
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'neutral' THEN 0
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'negative' THEN -1
            ELSE 0
        END), 3) as avg_sentiment_score
    FROM restaurants r
    JOIN orders o ON r.restaurant_id = o.restaurant_id
    LEFT JOIN customer_reviews cr ON r.restaurant_id = cr.restaurant_id
    WHERE o.status = 'delivered'
    GROUP BY r.restaurant_name, r.cuisine_type
)
SELECT 
    restaurant_name,
    cuisine_type,
    total_orders,
    avg_order_value,
    total_revenue,
    unique_customers,
    avg_sentiment_score,
    AI_CLASSIFY(
        'llama3-8b',
        'Restaurant with ' || total_orders || ' orders, $' || avg_order_value || ' average order value, ' ||
        '$' || total_revenue || ' total revenue, ' || unique_customers || ' unique customers, ' ||
        'sentiment score ' || avg_sentiment_score,
        ['Top Performer', 'Strong Performer', 'Average Performer', 'Needs Improvement', 'Underperformer']
    ) as performance_category,
    AI_COMPLETE(
        'llama3-8b',
        'Based on this restaurant performance data: ' || total_orders || ' orders, $' || total_revenue || 
        ' revenue, ' || unique_customers || ' customers, sentiment score ' || avg_sentiment_score || 
        '. Provide 2-3 specific actionable recommendations for improvement in 75 words or less.'
    ) as improvement_recommendations
FROM restaurant_analysis
ORDER BY total_revenue DESC;
```

**Key Changes:**
1. **Fixed AI_SENTIMENT**: Extract sentiment from `:categories[0]:sentiment` and convert to numeric (-1, 0, 1)
2. **Added model parameters**: Both `AI_CLASSIFY` and `AI_COMPLETE` now have `'llama3-8b'` model parameter
3. **Proper aggregation**: Now `AVG()` works on numeric values instead of JSON objects


## 6. AI_COMPLETE: Marketing Content Generation

Use AI_COMPLETE to generate marketing content, promotional descriptions, and customer communications.


In [ ]:
-- Generate promotional content for top-performing restaurants
WITH top_restaurants AS (
    SELECT 
        r.restaurant_name,
        r.cuisine_type,
        COUNT(o.order_id) as order_count,
        ROUND(AVG(o.total_amount), 2) as avg_order_value
    FROM restaurants r
    JOIN orders o ON r.restaurant_id = o.restaurant_id
    WHERE o.status = 'delivered'
    GROUP BY r.restaurant_name, r.cuisine_type
    ORDER BY order_count DESC
    LIMIT 5
)
SELECT 
    restaurant_name,
    cuisine_type,
    order_count,
    avg_order_value,
    AI_COMPLETE(
        'llama3-8b',
        'Write a compelling promotional email subject line for ' || restaurant_name || 
        ', a popular ' || cuisine_type || ' restaurant with an average order value of $' || avg_order_value ||
        '. Make it engaging and highlight their popularity.'
    ) as email_subject_line,
    AI_COMPLETE(
        'llama3-8b',
        'Create a 50-word promotional description for ' || restaurant_name || 
        ', highlighting their ' || cuisine_type || ' cuisine and popularity among customers. ' ||
        'Mention they have served ' || order_count || ' orders. Make it appetizing and compelling.'
    ) as promotional_description
FROM top_restaurants;

-- Business Insight: AI-generated marketing content scales promotional efforts 
-- and ensures consistent, engaging messaging across restaurant partners.


In [ ]:
-- Generate personalized customer retention messages
WITH customer_insights AS (
    SELECT 
        c.customer_id,
        c.first_name,
        c.loyalty_tier,
        c.preferred_cuisine,
        c.total_orders,
        c.total_spent,
        DATEDIFF(days, c.last_order_date, CURRENT_DATE()) as days_since_last_order
    FROM customers c
    WHERE days_since_last_order > 30  -- Customers who haven't ordered recently
    ORDER BY total_spent DESC
    LIMIT 10
)
SELECT 
    customer_id,
    first_name,
    loyalty_tier,
    preferred_cuisine,
    days_since_last_order,
    AI_COMPLETE(
        'llama3-8b',
        'Write a personalized win-back email for ' || first_name || 
        ', a ' || loyalty_tier || ' tier customer who loves ' || preferred_cuisine || ' food. ' ||
        'They have placed ' || total_orders || ' orders and spent $' || total_spent || ' total. ' ||
        'They havent ordered in ' || days_since_last_order || ' days. ' ||
        'Make it warm, personal, and include a compelling offer. Keep it under 100 words.'
    ) as retention_message
FROM customer_insights;

-- Business Insight: Personalized retention messages improve customer reactivation rates 
-- and demonstrate the value of AI in customer relationship management.


## 7. Advanced Analytics: Combining Multiple AISQL Functions

Let's combine multiple AISQL functions to create comprehensive business intelligence dashboards.


In [ ]:
-- Comprehensive restaurant performance analysis combining multiple AI functions
WITH restaurant_analysis AS (
    SELECT 
        r.restaurant_name,
        r.cuisine_type,
        COUNT(DISTINCT o.order_id) as total_orders,
        ROUND(AVG(o.total_amount), 2) as avg_order_value,
        ROUND(SUM(o.total_amount), 2) as total_revenue,
        COUNT(DISTINCT o.customer_id) as unique_customers,
        ROUND(AVG(CASE 
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'positive' THEN 1
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'neutral' THEN 0
            WHEN AI_SENTIMENT(cr.review_text):categories[0]:sentiment = 'negative' THEN -1
            ELSE 0
        END), 3) as avg_sentiment_score
    FROM restaurants r
    JOIN orders o ON r.restaurant_id = o.restaurant_id
    LEFT JOIN customer_reviews cr ON r.restaurant_id = cr.restaurant_id
    WHERE o.status = 'delivered'
    GROUP BY r.restaurant_name, r.cuisine_type
)
SELECT 
    restaurant_name,
    cuisine_type,
    total_orders,
    avg_order_value,
    total_revenue,
    unique_customers,
    avg_sentiment_score,
    AI_CLASSIFY(
        'Restaurant with ' || total_orders || ' orders, $' || avg_order_value || ' average order value, ' ||
        '$' || total_revenue || ' total revenue, ' || unique_customers || ' unique customers, ' ||
        'sentiment score ' || avg_sentiment_score,
        ['Top Performer', 'Strong Performer', 'Average Performer', 'Needs Improvement', 'Underperformer']
    ) as performance_category,
    AI_COMPLETE(
        'llama3-8b',
        'Based on this restaurant performance data: ' || total_orders || ' orders, $' || total_revenue || 
        ' revenue, ' || unique_customers || ' customers, sentiment score ' || avg_sentiment_score || 
        '. Provide 2-3 specific actionable recommendations for improvement in 75 words or less.'
    ) as improvement_recommendations
FROM restaurant_analysis
ORDER BY total_revenue DESC;

-- Business Insight: This comprehensive analysis combines sentiment analysis, 
-- classification, and content generation to provide actionable business intelligence.


## 3. AI_EXTRACT: Cleaning Up Messy Restaurant Data

Real-world restaurant data is often messy and unstructured. AI_EXTRACT can transform chaotic text into clean, structured data for analysis.

### 📋 **Data Loading Options:**

**For Quick Demo:** Use Option B (VALUES statements) - data loads instantly  
**For Realistic Demo:** Use Option A (CSV files) - shows real-world file loading

**If using CSV files, first upload these files to your stage:**
1. `messy_restaurant_data.csv` 
2. `messy_customer_reviews.csv`

Let's load some messy restaurant data that showcases AI_EXTRACT's power:
- **AI_SENTIMENT** enables real-time monitoring of customer satisfaction across all restaurant partners
- Automated sentiment scoring helps identify underperforming restaurants before they impact the platform
- Sentiment trends can predict customer churn and inform retention strategies

### 2. Operational Optimization
- **AI_CLASSIFY** categorizes menu items and customer segments for targeted marketing
- **AI_EXTRACT** pulls specific operational insights from unstructured data
- **AI_AGG** identifies patterns across large datasets that would be impossible to detect manually

### 3. Personalization at Scale
- **AI_EMBED** and **AI_SIMILARITY** power recommendation engines that increase order values
- Personalized recommendations improve customer experience and platform stickiness
- Similarity analysis helps restaurants understand competitive positioning

### 4. Marketing Automation
- **AI_COMPLETE** generates personalized marketing content at scale
- Automated content creation ensures consistent messaging while reducing manual effort
- Personalized retention campaigns improve customer lifetime value

### 5. Executive Decision Support
- **AI_SUMMARIZE_AGG** provides executive-level insights without manual analysis
- Combined AI functions create comprehensive business intelligence dashboards
- Data-driven recommendations support strategic decision-making

### ROI Impact for Olo:
- **Reduced churn** through proactive sentiment monitoring
- **Increased order values** via personalized recommendations
- **Improved operational efficiency** through automated insights
- **Enhanced customer experience** through AI-powered personalization
- **Scalable marketing** through automated content generation

This demonstration shows how AISQL functions transform raw operational data into actionable business intelligence, enabling Olo to compete more effectively in the competitive hospitality technology market.


In [ ]:
-- SIMPLE SOLUTION: Create the messy data directly (works immediately)
CREATE OR REPLACE TABLE messy_restaurant_data AS
SELECT * FROM VALUES
(1, 'RESTAURANT INFO: Tony''s Pizza Place!!! Located somewhere on 123 main street seattle wa phone is (206)555-1234 or maybe (206)555-1235?? HOURS: mon-fri 11am to 10pm, weekends 12pm-11pm. We serve AMAZING pizza, pasta, salads & more! Owner: Tony Marinelli since 2019. DELIVERY FEE $3.99 minimum order $15 CASH OR CARD ACCEPTED'),
(2, 'Dragon Wok Chinese Restaurant - Address: 456 Pine Ave, Seattle WA 98102 | Phone: 206.555.9876 | Email: info@dragonwok.com | SPECIALTIES: Kung Pao Chicken $12.95, Sweet & Sour Pork $11.50, Fried Rice $8.99, Lo Mein $9.99 | HOURS: Daily 11:30AM-9:30PM except Tuesdays CLOSED | Owner: Mrs. Chen | Est. 2020 | FREE delivery over $25!'),
(3, 'Burger Palace!!! Best burgers in town!!! 789 Oak Street Bellevue WA call us at 425-555-0199 OPEN everyday 10am-midnight Classic Burger $8.99 Cheeseburger $9.99 Bacon Burger $11.99 Fries $3.99 Shakes $4.99 Owner Mike Johnson Family owned since 2018 WE DELIVER! $2.99 delivery fee minimum $12 order'),
(4, 'TACO LOCO - Mexican Food - 321 Cedar St Redmond WA 98052 - Phone (425) 555-7777 - MENU: Tacos $2.50 each, Burritos $7.99, Quesadillas $6.99, Nachos $8.99 - HOURS: Mon-Thu 11am-9pm, Fri-Sat 11am-10pm, Sun 12pm-8pm - Carlos Rodriguez owner - CATERING AVAILABLE - Delivery $3.50 min order $10'),
(5, 'Sushi Zen Japanese Restaurant located at 654 Maple Dr Kirkland WA 98033 phone number is 425-555-8888 email orders@sushizen.com FRESH SUSHI DAILY! California Roll $6.99 Salmon Roll $8.99 Tuna Roll $9.99 Sashimi Combo $15.99 HOURS Tuesday-Sunday 5pm-10pm CLOSED MONDAYS Owner Yuki Tanaka since 2021 Delivery available $5.99 fee minimum $20')
AS t(messy_data_id, raw_text_data);

-- Let's see what this messy data looks like
SELECT messy_data_id, LEFT(raw_text_data, 100) || '...' as sample_text 
FROM messy_restaurant_data;


### Example 1: Extract Basic Restaurant Information

Now let's use AI_EXTRACT to clean up this messy data and extract structured information:


In [ ]:
-- AI_EXTRACT transforms messy text into clean, structured data
SELECT 
    messy_data_id,
    AI_EXTRACT(
        text => raw_text_data,
        responseFormat => [
            ['restaurant_name', 'What is the name of the restaurant?'],
            ['address', 'What is the full address?'],
            ['phone', 'What is the phone number?'],
            ['cuisine_type', 'What type of cuisine do they serve?'],
            ['owner', 'Who is the owner?']
        ]
    ) as extracted_info
FROM messy_restaurant_data
LIMIT 3;

-- Notice how AI_EXTRACT handles:
-- - Inconsistent formatting (ALL CAPS, mixed case, punctuation)
-- - Multiple phone numbers (picks the correct one)
-- - Scattered information throughout the text
-- - Different text structures and styles


### Example 2: Extract Menu Items and Pricing

Let's extract menu information from the same messy data:


In [ ]:
-- Extract menu items and pricing from messy restaurant descriptions
SELECT 
    messy_data_id,
    AI_EXTRACT(
        text => raw_text_data,
        responseFormat => [
            ['menu_items', 'List all menu items mentioned with their prices'],
            ['delivery_fee', 'What is the delivery fee?'],
            ['minimum_order', 'What is the minimum order amount?'],
            ['hours', 'What are the operating hours?']
        ]
    ) as menu_pricing_info
FROM messy_restaurant_data
WHERE messy_data_id IN (2, 3, 4);  -- Focus on restaurants with detailed menu info

-- AI_EXTRACT intelligently parses:
-- - Menu items scattered throughout text
-- - Inconsistent price formatting ($12.95 vs $8.99)
-- - Mixed delivery policies and fees
-- - Various hour formats (11:30AM-9:30PM vs 11am-9pm)


### Example 3: Extract Customer Review Insights from Messy Text

Now let's work with messy customer reviews:


In [ ]:
-- Option A: Load messy customer reviews from CSV
CREATE OR REPLACE TABLE messy_customer_reviews (
    review_id INTEGER,
    messy_review_text STRING
);

COPY INTO messy_customer_reviews
FROM @olo_data_stage/messy_customer_reviews.csv
FILE_FORMAT = (
    TYPE = 'CSV' 
    SKIP_HEADER = 1
    FIELD_OPTIONALLY_ENCLOSED_BY = '"'
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE
);

-- Option B: Quick demo version - create directly with VALUES
/*
CREATE OR REPLACE TABLE messy_customer_reviews AS
SELECT * FROM VALUES
(1, 'OMG just ordered from Tony''s Pizza Place last night and WOW!!! The margherita pizza was AMAZING - fresh basil, perfect crust, cheese was melted just right. Service was kinda slow though took like 45 mins but worth the wait I guess. Definitely ordering again! Oh and the garlic bread was good too. 4/5 stars would recommend to friends!'),
(2, 'Dragon Wok delivery was terrible!!! Food took 1.5 hours to arrive and when it finally got here the lo mein was cold and soggy :( The kung pao chicken was way too spicy even though I asked for mild. Only good thing was the fried rice which was actually pretty tasty. Won''t be ordering again. 2 out of 5 stars. Very disappointed.'),
(3, 'BURGER PALACE IS THE BEST!!! Had the bacon cheeseburger combo with fries and a chocolate shake. Burger was juicy and cooked perfectly, fries were crispy, shake was thick and creamy. Staff was super friendly too! Only complaint is it''s a bit pricey but you get what you pay for. Will definitely be back! 5 stars!')
AS t(review_id, messy_review_text);
*/

-- Extract structured insights from chaotic review text
SELECT 
    review_id,
    AI_EXTRACT(
        text => messy_review_text,
        responseFormat => [
            ['restaurant_name', 'What restaurant is being reviewed?'],
            ['items_ordered', 'What specific items did they order?'],
            ['positive_feedback', 'What did they like about their experience?'],
            ['negative_feedback', 'What complaints or issues did they mention?'],
            ['rating', 'What rating did they give (if mentioned)?'],
            ['delivery_time', 'How long did delivery take (if mentioned)?']
        ]
    ) as review_insights
FROM messy_customer_reviews;


### Example 4: Business Intelligence from Messy Data

Let's combine AI_EXTRACT with other AI functions for comprehensive analysis:


In [ ]:
-- Advanced: Combine AI_EXTRACT with AI_SENTIMENT and AI_CLASSIFY for comprehensive insights
WITH extracted_reviews AS (
    SELECT 
        review_id,
        messy_review_text,
        AI_EXTRACT(
            text => messy_review_text,
            responseFormat => [
                ['restaurant_name', 'What restaurant is being reviewed?'],
                ['overall_experience', 'Summarize their overall experience in one sentence'],
                ['key_issues', 'What were the main problems mentioned?'],
                ['rating', 'What rating did they give?']
            ]
        ) as extracted_data
    FROM messy_customer_reviews
)
SELECT 
    review_id,
    extracted_data:restaurant_name::STRING as restaurant_name,
    extracted_data:overall_experience::STRING as experience_summary,
    extracted_data:key_issues::STRING as main_issues,
    extracted_data:rating::STRING as customer_rating,
    
    -- Add sentiment analysis
    AI_SENTIMENT(messy_review_text):categories[0]:sentiment::STRING as sentiment,
    AI_SENTIMENT(messy_review_text):categories[0]:score::FLOAT as sentiment_score,
    
    -- Classify the review type
    AI_CLASSIFY(
        messy_review_text,
        ['Excellent Experience', 'Good Experience', 'Average Experience', 'Poor Experience', 'Terrible Experience']
    ) as experience_category,
    
    -- Generate action items
    AI_COMPLETE(
        'llama3-8b',
        'Based on this customer review: "' || LEFT(messy_review_text, 200) || '..." provide 2 specific action items for the restaurant to improve. Keep response under 50 words.'
    ) as action_items
    
FROM extracted_reviews;

-- This demonstrates the power of combining AI functions:
-- 1. AI_EXTRACT structures the messy text
-- 2. AI_SENTIMENT analyzes emotional tone
-- 3. AI_CLASSIFY categorizes the experience
-- 4. AI_COMPLETE generates actionable recommendations


### Key AI_EXTRACT Capabilities Demonstrated:

**🧹 Data Cleaning Power:**
- Handles inconsistent formatting (ALL CAPS, mixed case, excessive punctuation)
- Extracts structured data from completely unorganized text
- Identifies relevant information scattered throughout messy descriptions

**🎯 Smart Information Extraction:**
- Distinguishes between multiple phone numbers to find the correct one
- Separates menu items from promotional text
- Identifies ratings in various formats (4/5, 2 out of 5 stars, 5 stars!)

**🔄 Real-World Applications:**
- **Restaurant Onboarding**: Clean up messy restaurant submissions
- **Review Analysis**: Structure chaotic customer feedback
- **Data Migration**: Transform legacy text data into structured formats
- **Social Media Monitoring**: Extract insights from unstructured social posts

**💡 Business Value:**
- Eliminates manual data entry and cleaning
- Enables analysis of previously unusable text data
- Scales data processing from hundreds to millions of records
- Provides consistent structure for downstream analytics

---

## Key Business Insights and Takeaways


### 1. Customer Satisfaction Intelligence
- **AI_SENTIMENT** enables real-time monitoring of customer satisfaction across all restaurant partners
- Automated sentiment scoring helps identify underperforming restaurants before they impact the platform
- Sentiment trends can predict customer churn and inform retention strategies

### 2. Operational Optimization  
- **AI_CLASSIFY** categorizes menu items and customer segments for targeted marketing
- **AI_EXTRACT** pulls specific operational insights from unstructured data
- **AI_AGG** identifies patterns across large datasets that would be impossible to detect manually

### 3. Personalization at Scale
- **AI_EMBED** and **AI_SIMILARITY** power recommendation engines that increase order values
- Personalized recommendations improve customer experience and platform stickiness
- Similarity analysis helps restaurants understand competitive positioning

### 4. Marketing Automation
- **AI_COMPLETE** generates personalized marketing content at scale
- Automated content creation ensures consistent messaging while reducing manual effort
- Personalized retention campaigns improve customer lifetime value

### 5. Executive Decision Support
- **AI_SUMMARIZE_AGG** provides executive-level insights without manual analysis
- Combined AI functions create comprehensive business intelligence dashboards
- Data-driven recommendations support strategic decision-making

### ROI Impact for Olo:
- **Reduced churn** through proactive sentiment monitoring
- **Increased order values** via personalized recommendations
- **Improved operational efficiency** through automated insights
- **Enhanced customer experience** through AI-powered personalization
- **Scalable marketing** through automated content generation

This demonstration shows how AISQL functions transform raw operational data into actionable business intelligence, enabling Olo to compete more effectively in the competitive hospitality technology market.
